In [1]:
import os 
os.chdir('../../')
print("Current working directory is now: ", os.getcwd())

import pandas as pd 
import numpy as np
import csv
from sklearn.linear_model import LogisticRegression

import ky_utils.ky_interpretable_functions as interpret
import ky_utils.ky_riskslim as slim
import ky_utils.ky_stumps as stumps

from pprint import pprint
from riskslim.helper_functions import load_data_from_csv, print_model
from sklearn.metrics import roc_auc_score

Current working directory is now:  C:\Users\binha\Documents\Duke\Cynthia Research\interpretable-machine-learning\ky-fl-combined\ky-model


C:\Users\binha\Anaconda3\lib\site-packages\sklearn\externals\six.py:31: DeprecationWarning:

The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).



### EBM & CART

In [2]:
KY_data = pd.read_csv("~/Documents/Duke/Cynthia Research/data/ky-fl-data/KY-cleaned.csv").sort_values('person_id')
FL_data = pd.read_csv("~/Documents/Duke/Cynthia Research/data/ky-fl-data/FL-cleaned.csv").sort_values('person_id')

## split x and y 
KY_X = KY_data.loc[:,:'current_violence20']
KY_Y = KY_data['drug_two_year'].values
FL_X = FL_data.loc[:,:'current_violence20']
FL_Y = FL_data['drug_two_year'].values

In [3]:
#### CART
depth = [5,6,7,8,9,10]
cart_summary = interpret.CART(KY_X, KY_Y, FL_X, FL_Y, depth, seed=816)

### EBM
estimators = [40]
depth = [2]
learning_rate = [0.3]
ebm_summary = interpret.EBM(KY_X, KY_Y, FL_X, FL_Y, learning_rate, depth, estimators, seed=816)

In [4]:
np.mean(cart_summary['FL_score']), np.mean(ebm_summary['FL_score'])

(0.6180146602832833, 0.6346079428472204)

### Lasso Stumps

In [5]:
KY_stumps = pd.read_csv("~/Documents/Duke/Cynthia Research/data/ky-fl-data/KY-stumps.csv").sort_values('person_id')
FL_stumps = pd.read_csv("~/Documents/Duke/Cynthia Research/data/ky-fl-data/FL-stumps.csv").sort_values('person_id')

## split x and y 
KY_X = KY_stumps.loc[:,:'current_violence201']
KY_Y = KY_stumps['drug_two_year'].values
FL_X = FL_stumps.loc[:,:'current_violence201']
FL_Y = FL_stumps['drug_two_year'].values

## columns 
cols = KY_X.columns[1:]

In [7]:
single_stump_model = stumps.stump_model(KY_X, KY_Y, FL_X, FL_Y, 0.003, cols, 816)
single_stump_model['features']

['sex1',
 'age_at_current_charge19',
 'age_at_current_charge20',
 'age_at_current_charge24',
 'age_at_current_charge30',
 'age_at_current_charge36',
 'age_at_current_charge40',
 'age_at_current_charge43',
 'age_at_current_charge47',
 'p_arrest1',
 'p_arrest2',
 'p_arrest3',
 'p_arrest4',
 'p_arrest5',
 'p_charges1',
 'p_felony1',
 'p_felony2',
 'p_felony3',
 'p_property1',
 'p_weapon1',
 'p_drug1',
 'p_drug2',
 'p_drug3',
 'p_fta_two_year1',
 'p_fta_two_year2',
 'p_fta_two_year_plus1',
 'p_pending_charge1',
 'p_probation1',
 'p_incarceration1',
 'six_month1',
 'one_year1']

In [8]:
c_grid={'C': [0.001, 0.002, 0.003]}
stumps_summary = stumps.stump_cv(KY_X, KY_Y, FL_X, FL_Y, cols, c_grid, seed=816)

In [9]:
stumps_summary['best_params'], np.mean(stumps_summary['FL_score']), np.mean(stumps_summary['auc_diff'])

([{'C': 0.003}, {'C': 0.003}, {'C': 0.003}, {'C': 0.003}, {'C': 0.003}],
 0.6264855629697617,
 0.0006366061579327287)

### RiskSLIM

In [10]:
## train on best param chosen by Lasso Stumps from above
single_stump_model = stumps.stump_model(KY_X, 
                                        KY_Y, 
                                        FL_X, 
                                        FL_Y, 
                                        0.0008, 
                                        cols, 
                                        816)
selected_features = single_stump_model['features']
len(selected_features)

16

In [11]:
### Subset features
sub_FL_X = FL_stumps.loc[:, selected_features]
sub_KY_X = KY_stumps.loc[:, selected_features]
sub_KY_X.insert(0, '(Intercept)', 1)

In [12]:
riskslim_summary = slim.risk_cv(sub_KY_X, 
                                KY_Y, 
                                sub_FL_X, 
                                FL_Y, 
                                y_label = 'drug_two_year', 
                                max_coef = 5, 
                                max_coef_number = 5, 
                                max_runtime=1000, 
                                max_offset=100,
                                c=1e-6, 
                                seed=816)

setting c0 = 0.0 to ensure that intercept is not penalized
02/15/20 @ 06:59 PM | 213 rows in lookup table
02/15/20 @ 06:59 PM | ------------------------------------------------------------
02/15/20 @ 06:59 PM | runnning initialization procedure
02/15/20 @ 06:59 PM | ------------------------------------------------------------
02/15/20 @ 06:59 PM | CPA produced 2 cuts
02/15/20 @ 06:59 PM | running naive rounding on 3 solutions
02/15/20 @ 06:59 PM | best objective value: 0.6931
02/15/20 @ 06:59 PM | rounding produced 3 integer solutions
02/15/20 @ 06:59 PM | best objective value is 0.6931
02/15/20 @ 06:59 PM | running sequential rounding on 3 solutions
02/15/20 @ 06:59 PM | best objective value: 0.6931
02/15/20 @ 06:59 PM | sequential rounding produced 0 integer solutions
02/15/20 @ 06:59 PM | polishing 3 solutions
02/15/20 @ 06:59 PM | best objective value: 0.6931
02/15/20 @ 06:59 PM | polishing produced 3 integer solutions
02/15/20 @ 06:59 PM | initialization produced 5 feasible soluti

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
02/15/20 @ 06:59 PM | adding 279 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.2912.
Tried aggregator 1 time.
Reduced MIP has 34 rows, 36 columns, and 99 nonzeros.
Reduced MIP has 16 binaries, 18 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.05 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.04 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variabl

 137035 40124        0.2799    12        0.2855        0.2782   789693    2.54%           rho_7 U 137035 137034     21
 140197 40776        cutoff              0.2855        0.2783   801593    2.52%        alpha_13 U 140197 140195     34
 143192 41210        0.2783    12        0.2855        0.2783   814145    2.50%           rho_7 U 143192 143184     41
 146300 41892        0.2795    12        0.2855        0.2784   826674    2.48%           rho_4 D 146300 146299     15
 149410 42521        cutoff              0.2855        0.2784   839145    2.46%           rho_0 D 149410  16675     23
 152291 42677        cutoff              0.2855        0.2785   851721    2.42%        alpha_13 U 152291 152290     32
 155299 43249        cutoff              0.2855        0.2786   864051    2.40%           rho_2 D 155299 155298     47
 158209 43651        0.2821    10        0.2855        0.2787   876376    2.37%        alpha_13 D 158209 158208     35
 161169 43951        0.2851     6        0.2855 

02/15/20 @ 07:01 PM | ------------------------------------------------------------
02/15/20 @ 07:01 PM | runnning initialization procedure
02/15/20 @ 07:01 PM | ------------------------------------------------------------
02/15/20 @ 07:01 PM | CPA produced 2 cuts
02/15/20 @ 07:01 PM | running naive rounding on 3 solutions
02/15/20 @ 07:01 PM | best objective value: 0.6931
02/15/20 @ 07:01 PM | rounding produced 3 integer solutions
02/15/20 @ 07:01 PM | best objective value is 0.6931
02/15/20 @ 07:01 PM | running sequential rounding on 3 solutions
02/15/20 @ 07:01 PM | best objective value: 0.6931
02/15/20 @ 07:01 PM | sequential rounding produced 0 integer solutions
02/15/20 @ 07:01 PM | polishing 3 solutions
02/15/20 @ 07:01 PM | best objective value: 0.6931
02/15/20 @ 07:01 PM | polishing produced 3 integer solutions
02/15/20 @ 07:01 PM | initialization produced 5 feasible solutions
02/15/20 @ 07:01 PM | best objective value: 0.2892
02/15/20 @ 07:01 PM | -----------------------------

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
02/15/20 @ 07:01 PM | adding 258 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.2892.
Tried aggregator 1 time.
Reduced MIP has 34 rows, 36 columns, and 99 nonzeros.
Reduced MIP has 16 binaries, 18 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.05 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.04 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variabl

...

 143702 31345        cutoff              0.2837        0.2773   831331    2.28%           rho_7 U 143702 143701     21
 146657 31697        0.2803    10        0.2837        0.2773   844154    2.26%           rho_7 U 146657 146655     21
 149478 31738        0.2806     6        0.2837        0.2775   856259    2.22%           rho_8 U 149478 149476     34
 152325 32062        0.2775     1        0.2837        0.2775   868156    2.18%           rho_3 D 152325 152323     48
 155400 32613        0.2776     7        0.2837        0.2776   879591    2.17%           rho_7 U 155400 155386     47
 158285 33008        0.2805     8        0.2837        0.2776   890976    2.15%          rho_11 D 158285 158284     23
 161153 33212        0.2835     9        0.2837        0.2777   901539    2.12%          rho_11 D 161153 161152     30
 164248 33816        0.2801    10        0.2837        0.2778   913013    2.11%           rho_1 U 164248 164246     51
 167072 34027        cutoff              0.2837 

02/15/20 @ 07:56 PM | polishing 3 solutions
02/15/20 @ 07:56 PM | best objective value: 0.6931
02/15/20 @ 07:56 PM | polishing produced 3 integer solutions
02/15/20 @ 07:56 PM | initialization produced 5 feasible solutions
02/15/20 @ 07:56 PM | best objective value: 0.2900
02/15/20 @ 07:56 PM | ------------------------------------------------------------
02/15/20 @ 07:56 PM | completed initialization procedure
02/15/20 @ 07:56 PM | ------------------------------------------------------------
02/15/20 @ 07:56 PM | 213 rows in lookup table
CPXPARAM_Read_DataCheck                          1
CPXPARAM_Threads                                 1
CPXPARAM_Parallel                                1
CPXPARAM_RandomSeed                              0
CPXPARAM_TimeLimit                               1000
CPXPARAM_MIP_Tolerances_LowerCutoff              0.27468772830209653
CPXPARAM_MIP_Tolerances_UpperCutoff              0.28996813333638394


Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
02/15/20 @ 07:56 PM | adding 250 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.2900.
Tried aggregator 1 time.
Reduced MIP has 34 rows, 36 columns, and 99 nonzeros.
Reduced MIP has 16 binaries, 18 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.05 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.04 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variabl

 128916 42008        0.2757    14        0.2840        0.2751   892268    3.13%          rho_10 U 128916 106852     26
 132037 43135        0.2828     6        0.2840        0.2751   905626    3.12%         alpha_4 U 132037 132036     39
 135129 43918        0.2775    18        0.2840        0.2752   918585    3.10%          rho_10 U 135129 135128     26
 138160 44596        0.2783    11        0.2840        0.2752   931598    3.07%           rho_4 U 138160 138159     27
 141062 45386        0.2815    11        0.2840        0.2753   945316    3.05%           rho_5 D 141062 141061     39
 144087 46115        0.2754    12        0.2840        0.2754   958490    3.01%           rho_0 U 144087 144082     23
 147082 46872        0.2822    13        0.2840        0.2755   972616    2.99%           rho_7 D 147082 147081     37
 150166 47550        cutoff              0.2840        0.2755   985887    2.97%           rho_2 D 150166 150165     47
 153150 48149        0.2769    11        0.2840 

 460459 37333        cutoff              0.2840        0.2814  2249997    0.88%           rho_7 U 460459  57386     37
 471052 33022        0.2821    13        0.2840        0.2817  2289520    0.79%          rho_11 U 471052 168196     27
 481662 27930        cutoff              0.2840        0.2820  2327083    0.69%           rho_3 D 481662 481661     22
 491926 22139        cutoff              0.2840        0.2824  2362362    0.56%           rho_9 D 491926 269328     20
Elapsed time = 151.95 sec. (166775.26 ticks, tree = 10.87 MB, solutions = 8)
 502481 15823        cutoff              0.2840        0.2828  2395211    0.42%          rho_16 U 502481 379086     33
 512976  8650        cutoff              0.2840        0.2832  2424585    0.26%          rho_10 U 512976 223903     43

Gomory fractional cuts applied:  1
User cuts applied:  599

Root node processing (before b&c):
  Real time             =    0.10 sec. (1.65 ticks)
Sequential b&c:
  Real time             =  160.30 sec. (17818

In [13]:
np.mean(riskslim_summary['KY_validation']), np.mean(riskslim_summary['FL_score'])

(0.6983800784089091, 0.6234186242651254)

In [14]:
#### save results
summary_drug2_ky_inter_model = {"cart": cart_summary,
                                "ebm": ebm_summary,
                                "stumps": stumps_summary,
                                "riskslim": riskslim_summary}

In [15]:
path = "./results/interpretable/two-year/"
results = [["Drug",             
            np.str(round(np.mean(cart_summary['FL_score']),3)) + " (" + np.str(round(np.std(cart_summary['FL_score']),3)) + ")", 
            np.str(round(np.mean(ebm_summary['FL_score']), 3)) + " (" + np.str(round(np.std(ebm_summary['FL_score']),3)) + ")", 
            np.str(round(np.mean(stumps_summary['FL_score']),3)) + " (" + np.str(round(np.std(stumps_summary['FL_score']),3)) + ")", 
            np.str(round(np.mean(riskslim_summary['FL_score']), 3)) + " (" + np.str(round(np.std(riskslim_summary['FL_score']),3)) + ")"]]
with open(path + 'two-year-ky-interpretable.csv', 'a') as writeFile:
    writer = csv.writer(writeFile)
    writer.writerows(results)